In [2]:
import os
import numpy as np
import pandas as pd
import IPython.display as ipd
import tgt
import math
import sklearn
import librosa
import re
import scipy.io.wavfile
import parselmouth
from numpy import unique
from numpy import where
from matplotlib import pyplot as plt
import librosa.display
from sklearn import metrics
from scipy.signal import lfilter, hamming
from textGrid_AudioTrim import readTextGridUpdate,audio_trimming
%matplotlib inline

In [3]:
def formantLpc(newWavFile_vowel):
    #preprocessing first step -->get the hamming window 
    x=newWavFile_vowel
    N = len(x)
    w = np.hamming(N)
    #preprocessing step 2 --> apply the window and the high pass filter 
    # Apply window and high pass filter.
    #issues to be fixed in this version of code is  
    #a) how to choose the order in the lpc 
    #b) how to filter out the resultant frequency array 
    # c) how many formants to get 
    x1 = x * w
    x1 = lfilter([1], [1., 0.63], x1)
    Fs =16000
    ncoeff = 2 + Fs / 1000
    print(ncoeff)
    A = librosa.core.lpc(x1, int(ncoeff))
    rts = np.roots(A)
    rts = rts[np.imag(rts) >= 0]
    angz = np.arctan2(np.imag(rts), np.real(rts))
    frqs = angz * (Fs / (2 *  np.pi))
    frqs.sort()
    
    frqs=[i for i in frqs if(i>0.0 and i<=5500)]
    
    if(len(frqs)>=3):
        return frqs[0:3]
    else:
        return frqs
    




In [4]:
def calculateFormantsPraat(formant,atTime):
    formants=[]
    for i in range(1,6):
      #  print(i)
        x=formant.get_value_at_time(i,atTime)
        if(x!=0 and np.isnan(x)==False):
            formants.append(x)
    print(formants)
    formants.sort()
    if(len(formants)>=3):
        return formants[0:3]
    else:
        return formants

In [5]:
def getFormantsPraat(start_time,end_time,sound,formantType):
    point10=start_time+((end_time-start_time)*0.1)
    point20 = start_time + ((end_time-start_time)*0.2)
    point50 = start_time + ((end_time-start_time)*0.5)
    point80 = end_time - ((end_time-start_time)*0.2)
  
   # print("start and end time for formants",point20,point50,point80)
    formant = sound.to_formant_burg(max_number_of_formants=5, maximum_formant=5500)
    if(formantType==50):
        formants=calculateFormantsPraat(formant,point50)
        if(np.isnan(formants).all()==True):
           
            point50=point50+point10
        formants=calculateFormantsPraat(formant,point50)
        
    elif(formantType==20):
        formants=calculateFormantsPraat(formant,point20)
        if(np.isnan(formants).all()==True):
            
            point20=point20+point10
        formants=calculateFormantsPraat(formant,point20)
        
    
    elif(formantType==80):
        formants=calculateFormantsPraat(formant,point80)
        if(np.isnan(formants).all()==True):
           
            point80=point80+point10
        formants=calculateFormantsPraat(formant,point80)
        
    
    
    
    return formants


In [6]:
def extractAllFormants(final_files,base_path_textGrid,base_path_audio,word,pronunciation_vowel,formantType,formantMethod):
    formants_vowel=[]
    filenames=[]
    SAMPLE_RATE=16000
    i=0

    for path in final_files:
        filenames.append(path)
        
        try:
            print(i)
            i=i+1
     
            textGrid_file_path=os.path.join(base_path_textGrid,path)+'.TextGrid'
            start_time_word,end_time_word,start_time_pronunciation_vowel,end_time_pronunciation_vowel=readTextGridUpdate(textGrid_file_path,word,pronunciation_vowel)
            
           # print(start_time_word,end_time_word,start_time_pronunciation_vowel,end_time_pronunciation_vowel)
            audio_file_path=os.path.join(base_path_audio,path)+'.wav'
            sound=parselmouth.Sound(audio_file_path)
            
            if(formantMethod=='praat'):
                sound=parselmouth.Sound(audio_file_path)
                formants=getFormantsPraat(start_time_pronunciation_vowel,end_time_pronunciation_vowel,sound,formantType)
            
            elif(formantMethod=='lpc'):
                newWavFile_vowel=audio_trimming(audio_file_path,start_time_pronunciation_vowel,end_time_pronunciation_vowel)
                formants=formantLpc(newWavFile_vowel)
            
            formants_vowel.append(formants)
           # print(formants_vowel)
           # print(filenames)
            
        except Exception as e:
            print(e)
    
    
    df_vowel=pd.DataFrame({'feature':formants_vowel})
    df_vowel=df_vowel.feature.apply(pd.Series)
    df_vowel['file_name']=filenames
    
    return df_vowel